In [1]:
from database import *
from helpers import *
from reporter import *
import requests
import json

In [2]:
from datetime import timedelta, datetime

In [70]:
def ceiltime(dt, delta):
    return dt + (datetime.min - dt) % delta

In [71]:
start_delay = 1
logr_start_time = datetime.now() + timedelta(seconds=start_delay)

# We'll start the news logger when the next minute % 10 == 0 (e.g. XX:10:00, XX:20:00, ...)
news_start_time = ceiltime(logr_start_time, timedelta(minutes=5))



In [72]:
logr_start_time

datetime.datetime(2021, 7, 29, 12, 6, 51, 241523)

In [75]:
(datetime.min-logr_start_time) % timedelta(minutes=5)

datetime.timedelta(seconds=188, microseconds=758477)

In [78]:
(datetime.min-logr_start_time)

datetime.timedelta(days=-738000, seconds=42788, microseconds=758477)

In [ ]:
import pytz
pytz.all_timezones

In [14]:
Database.connect()

Connecting to databases... done!


In [1]:
from reporter import *

In [2]:
Reporter.send_report()

In [60]:
api_name = 'users'
data = Database.rt.child(api_name).get().val()

In [53]:
data = {"name": "Mortimer 'Morty' Smith"}
Database.rt.child("users").push(data)

{'name': '-MfGKmEi-PVFvMRj54Qo'}

In [54]:
data = {"name": "Mortimer 'Morty' Smith"}
Database.rt.child("users").child("Morty").set(data)


{'name': "Mortimer 'Morty' Smith"}

In [55]:
Database.rt.child("users").child("Morty").update({"name": "Mortiest Morty"})

{'name': 'Mortiest Morty'}

In [29]:
import gzip
import json
with gzip.GzipFile(name, "w") as outfile:
    outfile.write(json.dumps({ 'data' : data }).encode('utf-8'))

In [46]:
with gzip.GzipFile(name, "r") as outfile:
    s = json.loads(outfile.read().decode('utf-8'))

In [ ]:
Database.db.child(path).put(name)

In [61]:
data = Database.rt.child(api_name).get().val()
Database.rt.child(api_name).remove()            
date = localtime() #if offset is None else localtime() + offset
name = date.strftime("%Y-%m-%d") + "-{}.json".format(api_name)
path = "{}/{}/{}/{}".format(api_name, date.year, date.month, name)
with gzip.GzipFile(name, "w") as outfile:
    outfile.write(json.dumps({ 'data' : data }).encode('utf-8'))
Database.db.child(path).put(name)
os.remove(name)
Reporter.append('({}) {} log count: {}'.format(localtime(fmt=True), api_name, len(data) if data else 0))
if progress: print("Stored {} data!".format(api_name))

In [3]:
INTERVAL = 5   # Minimum number of minutes that must pass to retrieve news
ARTICLES = 5   # Number of articles to gather every INTERVAL minutes between [1, 50]
base_url = 'https://cloud.iexapis.com/stable/{}'
sand_url = 'https://sandbox.iexapis.com/stable/{}'
api_pkey = '?token=' + 'pk_18221745178442aa83406f8a3f877c2e'
api_skey = '?token=' + 'sk_a82b032b40ff4f3bb7824074dc9243d5'
logrname = 'News'
curr_url = base_url
progress = True

In [8]:
def _purl(endpoint):
    url = curr_url.format(endpoint)
    return url + api_pkey

def _surl(endpoint):
    url = curr_url.format(endpoint)
    return url + api_skey

def log():
    try:
        ltime = localtime(fmt=True)
        reqst = requests.get(_purl('stock/btcusd/news/last/{}/'.format(ARTICLES)))
        items = json.loads(reqst.content)
        items = { "LAtime" : ltime, "articles" : items }
        items = Database.rt.child(logrname).push(items)
        if progress: print("Logged {} data: {}".format(logrname, items))
    except Exception as e:
        Reporter.send_urgent(logrname, "logging", repr(e))

In [48]:
ltime = localtime("Canada/Eastern", fmt=True)

In [10]:
reqst = requests.get(_purl('stock/btcusd/news/last/{}/'.format(ARTICLES)))
items = json.loads(reqst.content)

In [31]:
items = json.loads(reqst.content)

In [ ]:
items

In [24]:
items = { "LAtime" : ltime, "articles" : items }

In [32]:
items = Database.rt.child(logrname).push(items)

In [29]:
items = Database.rt.child(logrname).push(items)

In [3]:
base_url = 'https://www.bitstamp.net/api/v2/ticker/btcusd/'
logrname = 'Stocks'

#base_url = 'https://www.bitstamp.net/api/v2/transactions/btcusd/?time=day'
#logrname = 'Transactions'

progress = True

In [8]:
index = 1
minute_list = [1]*4
minute_list[index] = 45000
minute_list[index-1] = 44000

fifteen_minutes_diff = minute_list[index] - minute_list[index-1]
fifteen_minutes_diff_percentage = fifteen_minutes_diff/minute_list[index-1]

if fifteen_minutes_diff < 0:
    msg = '15 minutes changes: decrese {:2.4f} percentage, 15 minutes ago price {}, current price {}'.format(fifteen_minutes_diff_percentage*100, minute_list[index-1], minute_list[index])
    #msg = msg_format
    Reporter.price_change_alert(msg)
    #print('15 minutes changes: decrese {:2.4f} percentage, 15 minutes ago price {}, current price {}'.format(fifteen_minutes_diff_percentage*100, minute_list[index-1], minute_list[index]))
    
if fifteen_minutes_diff > 0:
    msg = '15 minutes changes: increase {:2.4f} percentage, 15 minutes ago price {}, current price {}'.format(fifteen_minutes_diff_percentage*100, minute_list[index-1], minute_list[index])
    Reporter.price_change_alert(msg)
    #print('15 minutes changes: increase {:2.4f} percentage, 15 minutes ago price {}, current price {}'.format(fifteen_minutes_diff_percentage*100, minute_list[index-1], minute_list[index]))

In [ ]:
minute_list = [-1]*4
hour_list = [-1]*24

import time
while(1):
    ltime = localtime(fmt=True)
    time_obj = localtime()
    
    reqst = requests.get(base_url)
    items = json.loads(reqst.content)
    
    msg_format = '{} changes: {} {:2.4f} percentage, {} ago price {}, current price {}'
    decrease = 'decresse'
    increase = 'increase'
    
    if time_obj.second==0:
        if time_obj.minute%15==0:
            print('time: ', ltime)
            index = int(time_obj.minute/15)
            minute_list[index] = float(items['last'])
            if minute_list[index-1] != -1:
                fifteen_minutes_diff = minute_list[index] - minute_list[index-1]
                fifteen_minutes_diff_percentage = fifteen_minutes_diff/minute_list[index-1]
                print('fifteen minutes diff: ', fifteen_minutes_diff)
                if fifteen_minutes_diff < 0:
                    msg = msg_format.format('15 minutes', decrease, fifteen_minutes_diff_percentage*100, '15 minutes', minute_list[index-1], minute_list[index])
                    Reporter.price_change_alert(msg)
                    #print('15 minutes changes: decrese {:2.4f} percentage, 15 minutes ago price {}, current price {}'.format(fifteen_minutes_diff_percentage*100, minute_list[index-1], minute_list[index]))
                    
                if fifteen_minutes_diff > 0:
                    msg = msg_format.format('15 minutes', increase, fifteen_minutes_diff_percentage*100, '15 minutes', minute_list[index-1], minute_list[index])
                    Reporter.price_change_alert(msg)
                    #print('15 minutes changes: increase {:2.4f} percentage, 15 minutes ago price {}, current price {}'.format(fifteen_minutes_diff_percentage*100, minute_list[index-1], minute_list[index]))
                    
        if time_obj.minute==0:
            index = time_obj.hour 
            hour_list[index] = float(items['last'])
            if hour_list[index-1] != -1:
                one_hour_diff = hour_list[index] - hour_list[index-1]
                one_hour_diff_percentage = one_hour_diff/hour_list[index-1]
                print('one hour diff: ', one_hour_diff)
                if one_hour_diff < 0:
                    msg = msg_format.format('One hour', decrease, one_hour_diff_percentage*100, 'One hour', hour_list[index-1], hour_list[index])
                    Reporter.price_change_alert(msg)
                    #print('one hour changes: decrese {:2.4f} percentage, one hour ago price {}, current price {}'.format(one_hour_diff_percentage*100, hour_list[index-1], hour_list[index]))
                    
                if one_hour_diff > 0:
                    msg = msg_format.format('One hour', increase, one_hour_diff_percentage*100, 'One hour', hour_list[index-1], hour_list[index])
                    Reporter.price_change_alert(msg)
                    #print('one hour changes: increase {:2.4f} percentage, one hour ago price {}, current price {}'.format(one_hour_diff_percentage*100, hour_list[index-1], hour_list[index]))
                    
            if index%4==0 and hour_list[index-4]!=-1:
                four_hours_diff = hour_list[index] - hour_list[index-4]
                four_hours_diff_percentage = four_hours_diff/hour_list[index-4]
                print('four hours diff: ', four_hours_diff)
                if four_hours_diff < 0:
                    msg = msg_format.format('Four hours', decrease, four_hours_diff_percentage*100, 'Four hours', hour_list[index-1], hour_list[index])
                    #print('four hours changes: decrese {:2.4f} percentage, four hours ago price {}, current price {}'.format(four_hours_diff_percentage*100, hour_list[index-1], hour_list[index]))
                    
                if four_hours_diff > 0:
                    msg = msg_format.format('Four hours', increase, four_hours_diff_percentage*100, 'Four hours', hour_list[index-1], hour_list[index])
                    Reporter.price_change_alert(msg)
                    #print('four hours changes: increase {:2.4f} percentage, four hours ago price {}, current price {}'.format(four_hours_diff_percentage*100, hour_list[index-1], hour_list[index]))
    time.sleep(1)

In [36]:
ltime.hour
ltime.minute
type(ltime.second)

int

In [27]:
ltime.microsecond

12982

In [ ]:
from datetime import datetime
str(datetime.edcnow())

In [ ]:
ltime = localtime(fmt=True)
reqst = requests.get(base_url)
items = json.loads(reqst.content)

items

In [ ]:
items["LAtime"] = ltime
items = Database.rt.child(TckrLogger.logrname).push(items)
if TckrLogger.progress: print("Logged {} data: {}".format(TckrLogger.logrname, items))